In [1]:
import pickle
import glob
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
import lightgbm as lgb
from sklearn.metrics import f1_score
# import time
# print('sleeping')
# time.sleep(7200)
# print('sleep done =======================')
# load feats
train_x,test_x = [],[]
for feat in sorted(glob.glob('../feature/stacking_pkl_file/*')):
    if 'fm2.pkl' in feat or 'ligbm' in  feat :
        continue
    print('file path',feat)
    a,b = pickle.load(open(feat,'rb'))
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
    

file path ../feature/stacking_pkl_file/2leve_lgb_stacking3.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/Capsule.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/capsule_all_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/capsule_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/cnn.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/mlp.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/mlp_shuffle.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/pseudo_rnn2.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn7879.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_all_shuffle_all.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_cnn.pkl
(102277, 19) (102277, 19)
file path ../feature/stacking_pkl_file/rnn_pseudo1.pkl
(102277, 19) (102277, 19)
file path ../feature/stackin

In [2]:
for feat in sorted(glob.glob('../feature/other_features/features-vinson/*')):
    if 'fm2.pkl' in feat or 'w2v' in feat or 'lda' in feat or 'lsi' in feat:
        continue
    print('file path',feat)
    file=pd.read_csv(feat)
    a = file.iloc[:102277,:]
    b = file.iloc[102277:,:]
    a = a.values
    b = b.values
    print(a.shape,b.shape)
    train_x.append(a)
    test_x.append(b)
train_x = np.nan_to_num(np.hstack(train_x))
test_x = np.nan_to_num(np.hstack(test_x))
print(train_x.shape)

file path ../feature/other_features/features-vinson/bnb_prob_10w.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/lr_prob.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/lr_prob_0.778620.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/mnb_prob_10w.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/nn_ensemble_0.775597.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/svc_prob.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/svc_prob_0.778881.csv
(102277, 19) (102277, 19)
file path ../feature/other_features/features-vinson/word_fasttext_0.760225.csv
(102277, 19) (102277, 19)
(102277, 418)


In [3]:
import pandas as pd
import numpy as np
train = pd.read_csv("../input/new_data/train_set.csv")
y=(train["class"]-1).astype(int)
del train
import gc
gc.collect()

/usr/local/lib/python3.5/dist-packages/pandas/core/computation/check.py:17: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


7

In [4]:

from keras.utils import np_utils
train_y=np_utils.to_categorical(y,num_classes=19)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
def self_f1_score(data, y_hat):
    y_true = data
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return f1_score(y_true, y_hat, average='macro')

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat, average='macro'), True

In [54]:
def self_f1_score(data, y_hat):
    y_true = data
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return f1_score(y_true, y_hat, average='macro')

In [57]:
from sklearn.metrics import roc_auc_score
def fit_cv(estimator, params, train, test, y, label_id, n_splits=10):
    estimators,scores, test_predictions, f1_sco = [],[],[],[]
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=233)
    for train_index, test_index in kf.split(train_x, y[:, label_id]):
#         (X_train, y_train), (X_valid, y_valid), X_test = get_fold_xy(train, test, LABEL_COLUMNS,i)

        curr_x,curr_y = train[train_index],y[train_index]
        hold_out_x,hold_out_y = train[test_index],y[test_index]
        d_train = lgb.Dataset(curr_x, curr_y[:,label_id])
        d_valid = lgb.Dataset(hold_out_x, hold_out_y[:,label_id])
        watchlist = [d_train, d_valid]
        estimator_ = estimator
        model = estimator_.train(params, #                       feval=lgb_f1_score,
                      train_set=d_train,
                      num_boost_round=2000,
                      valid_sets=watchlist,
                      early_stopping_rounds=100,
                      verbose_eval=None)        
        y_valid_pred = model.predict(hold_out_x)
        y_test_pred = model.predict(test)
        score = roc_auc_score(hold_out_y[:,label_id], y_valid_pred)
#         curr_train_f1 = self_f1_score(curr_y[:,i],train_pred)
        curr_val_f1 = self_f1_score(hold_out_y[:,label_id],y_valid_pred)
        estimators.append(estimator_)
        scores.append(score)
        f1_sco.append(curr_val_f1)
        test_predictions.append(y_test_pred)
    return scores, estimators, test_predictions,f1_sco

In [60]:
from itertools import product
import lightgbm as lgb

estimator = lgb
def make_grid(param_grid):
    keys, values = zip(*param_grid.items())
    param_dicts = [dict(zip(keys, v)) for v in product(*values)]
    return param_dicts      
param_grid = dict(
                  metric=['binary_logloss'],
                    scale_pos_weight = [100],
                  num_leaves= [8, 12, 15, 16, 24],
                  max_depth = [3,4,5,6],
                  learning_rate= [0.03, 0.04, 0.05],
#                   max_depth= [2,3,4,5],
                  min_child_weight= [1,3,5,7],
#                   gamma=[0.01,0.05,0.1,0.5],
                  bagging_fraction= [1.0,0.8,0.6],
                  feature_fraction= [0.4,0.6,0.8,1.0], 
#                   reg_lambda= [0.0,0.01,0.1,0.5,1.0], #1.0
#                   reg_alpha= [0.0],
                  num_threads=[4]
                 )

label_id = 0
nr_runs = 50
grid_sample = np.random.choice(make_grid(param_grid), nr_runs, replace=False)


In [39]:
from tqdm import tqdm

In [61]:
from sklearn.model_selection import KFold, StratifiedKFold
grid_scores = []
for params in tqdm(grid_sample):
    scores, estimators, test_prediction, f1_sco = fit_cv(estimator, params, train_x, test_x, train_y,0, n_splits=10)  
    print('mean {} std {}'.format(np.mean(scores), np.std(scores)))
    print('mean {} std{}'.format(np.mean(f1_sco), np.std(f1_sco)))
    grid_scores.append((params, np.mean(scores), np.mean(f1_sco)))


  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:186: UserWarning: Usage subset(sliced data) of np.ndarray is not recommended due to it will double the peak memory cost in LightGBM.
  warnings.warn("Usage subset(sliced data) of np.ndarray is not recommended due to it will double the peak memory cost in LightGBM.")
/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
  2%|▏         | 1/50 [02:25<1:58:50, 145.53s/it]

mean 0.9355577547830819 std 0.044407811996916464
mean 0.7676253212151011 std0.09375661015108874


  4%|▍         | 2/50 [05:01<2:00:40, 150.83s/it]

mean 0.934934523952405 std 0.04466555741981172
mean 0.7677538036085533 std0.09386498517199761


  6%|▌         | 3/50 [07:20<1:54:57, 146.76s/it]

...

mean 0.9396376459563838 std 0.03177858982564365
mean 0.7685714483761711 std0.09412138906027824


 98%|█████████▊| 49/50 [3:16:27<04:00, 240.56s/it]

mean 0.8961828710558823 std 0.1609966899653193
mean 0.768348944720306 std0.09405614769383874


100%|██████████| 50/50 [3:19:24<00:00, 239.28s/it]

mean 0.8965589613208875 std 0.16112697081328753
mean 0.7679052151066272 std0.09391258708529736


In [63]:
    
best_params = sorted(grid_scores, key= lambda x: x[1])[-1]
print(best_params)

({'num_leaves': 8, 'max_depth': 5, 'verbose': 1, 'num_threads': 4, 'feature_fraction': 0.4, 'min_child_weight': 1, 'learning_rate': 0.03, 'scale_pos_weight': 100, 'metric': 'binary_logloss', 'bagging_fraction': 0.8}, 0.9482217748942322, 0.7675839237236142)


In [62]:
grid_scores

[({'bagging_fraction': 0.6,
   'feature_fraction': 0.8,
   'learning_rate': 0.05,
   'max_depth': 5,
   'metric': 'binary_logloss',
   'min_child_weight': 7,
   'num_leaves': 16,
   'num_threads': 4,
   'scale_pos_weight': 100,
   'verbose': 1},
  0.9355577547830819,
  0.7676253212151011),
 ({'bagging_fraction': 0.6,
   'feature_fraction': 0.8,
   'learning_rate': 0.05,
   'max_depth': 4,
   'metric': 'binary_logloss',
   'min_child_weight': 3,
   'num_leaves': 15,
   'num_threads': 4,
   'scale_pos_weight': 100,
   'verbose': 1},
  0.934934523952405,
  0.7677538036085533),
 ({'bagging_fraction': 0.6,
   'feature_fraction': 0.6,
   'learning_rate': 0.04,
   'max_depth': 4,
   'metric': 'binary_logloss',
   'min_child_weight': 7,
   'num_leaves': 12,
   'num_threads': 4,
   'scale_pos_weight': 100,
   'verbose': 1},
  0.939196269290916,
  0.7677342044867732),
 ({'bagging_fraction': 0.6,
   'feature_fraction': 0.8,
   'learning_rate': 0.03,
   'max_depth': 3,
   'metric': 'binary_logloss